In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import pandas as dd
import os

In [ ]:
class EvaluationDataset(Dataset):
    def __init__(self, directory,travel_time_directory,label_directory):
        
        self.files = [os.path.join(directory, f) for f in sorted(os.listdir(directory))] #[0:100]
        self.travel_time_files = [os.path.join(travel_time_directory, f) for f in sorted(os.listdir(travel_time_directory))]
        self.yfiles= [os.path.join(label_directory, f) for f in sorted(os.listdir(label_directory))] #[0:100]
        self.num_files = len(self.files)
        
        self.total_sequences = self.num_files # * self.sequences_per_file
        
    def __len__(self):
        return self.total_sequences
    
    def __getitem__(self, index):
        
        file_index = index 
        
        data = np.load(self.files[file_index]) #Loads the file which contains our desired sequence
        label_data = np.load(self.yfiles[file_index])
        travel_time_data = np.load(self.travel_time_files[file_index])

        sequence = np.transpose(data, (2, 0, 1)) # To orient in the dimension with 3 channels being first
        
        #When majority class is important
        # label_data_summed = np.sum(label_data,axis=0)
        # indx = np.argmax(label_data_summed)
        # labelsWindow = np.zeros(13)
        # labelsWindow[indx] = 1
        
        
        #When latest trend is important
        # last_row = label_data[-1:]
        # indx = np.argmax(last_row)
        # labelsWindow = np.zeros(13)
        # labelsWindow[indx] = 1
        
        # Just checking if it can predict accident or not based on latest row class
        last_row = label_data[-1:]
        indx = np.argmax(last_row)
        labelsWindow = np.zeros(2)
        if indx != 12:
            labelsWindow[0] = 1 # Accident true 
        else:
            labelsWindow[1] = 1 # Accident false
        
        
        # Travel Time lets take the last row row
        
        travel_time_data = travel_time_data[-1]
        
        return torch.tensor(sequence, dtype=torch.float32),torch.tensor(labelsWindow, dtype=torch.float32),torch.tensor(travel_time_data, dtype=torch.float32)

In [ ]:
#EXPERIMENT PARAMETERS

experiment_no = 11
eval_features_path = r"data/dataset4/features"
eval_labels_path = r"data/dataset4/labels"
eval_travel_time_path = r"data/dataset3/travel_time"
model_path = r""

In [ ]:
print(f"Is CUDA available : {torch.cuda.is_available()}")
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
model = torch.load(model_path)
model.to(device)


In [ ]:
dataset = EvaluationDataset(eval_features_path,eval_travel_time_path, eval_labels_path)
eval_dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0) # Shuffle is usually False for testing/validation

In [ ]:
history = [0] #intially there is no incident
incident_counter = 0
detected_counter = 0
new_incident_flag = False
incident_detected_flag = False
i = 0 #Time since last incident
false_alarm_counter = 0
detection_time = []


with torch.no_grad(): 
    
    for inputs, labels,travel_time in eval_dataloader:
        
        inputs, labels,travel_time = inputs.to(device), labels.to(device), travel_time.to(device)
        
        outputs = model(inputs,travel_time)
        
        _, predicted = torch.max(outputs.data, 1)
        _, labels_indices = torch.max(labels, 1)
        
        previous_label = history[-1]
        
        current_label = 1 if labels_indices == 0 else 0
        history.append(current_label)
        
        # New Incident Started    
        if previous_label == 0 and current_label == 1:
            
            incident_counter += 1
            new_incident_flag = True
            incident_detected_flag = False
            i = 0
        
        # Incident Detected
        if new_incident_flag and predicted == 0 and not incident_detected_flag:

            detected_counter += 1
            detection_time.append(i)
            incident_detected_flag = True            
            print(f"Incident Detected by alarm after {i}")
        
        # Incident Finished
        if previous_label == 1 and current_label == 0:
            new_incident_flag = False
            
        # Increment timer if incident ongoing but not yet detected
        if new_incident_flag and not incident_detected_flag:
            i+=1
            
        # False alarm raised   
        if not new_incident_flag and predicted == 0 and not (1 in history[-20:]):
            
            false_alarm_counter+=1
            print('False Alarm raised by model.')
        


In [ ]:
mean_detection_time = sum(detection_times) / len(detection_times) if detection_times else 0
print(f"Total Incidents: {incident_counter}, Detected Incidents: {detected_counter}, False Alarms: {false_alarm_counter}")
print(f"Mean Time to Detection: {mean_detection_time} timesteps")